# Symulacja 2D PDE (sim_02)

Notebook powstał na podstawie `sim_02_pde_slver.py`.

- Wszystkie wykresy oraz klatki animacji GIF mają tło **#252629**.
- Na początku znajduje się edytowalna ścieżka do katalogu wynikowego (wykresy/animacje).


In [ ]:
from pathlib import Path

# =============================
# USTAWIENIA UŻYTKOWNIKA
# =============================
# Edytuj tę ścieżkę, jeśli chcesz zapisywać wyniki w innym miejscu.
OUTPUT_DIR = Path('../figures/sim_01')

# Kolor tła dla wykresów i klatek GIF
BG_COLOR = '#252629'

# Tworzenie katalogów, jeśli nie istnieją
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FRAMES_DIR = OUTPUT_DIR / 'frames'
FRAMES_DIR.mkdir(parents=True, exist_ok=True)

print('Wyniki będą zapisywane do:', OUTPUT_DIR.resolve())
print('Klatki animacji będą zapisywane do:', FRAMES_DIR.resolve())


In [ ]:
import sys
from pathlib import Path

# =============================
# ŚCIEŻKI PROJEKTU / IMPORTY
# =============================
# Skrypt .py dodawał do sys.path katalog nadrzędny względem lokalizacji pliku.
# W notebooku najbezpieczniej ustawić jawnie katalog projektu.
# Jeśli importy nie działają, zmień PROJECT_ROOT.
PROJECT_ROOT = Path.cwd().resolve().parent

sys.path.insert(0, str(PROJECT_ROOT))
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio

# Importy symulacji
from src.pde_2d import SimConfig, Grid, run_2d_simulation

# Styl (jeśli dostępny w projekcie)
try:
    from src.viz_style import set_style, COLORS
    set_style()
except Exception:
    COLORS = {'pde': 'blue', 'dose': 'red'}

# Wymuszenie tła #252629 dla wszystkich figur i osi
plt.rcParams['figure.facecolor'] = BG_COLOR
plt.rcParams['axes.facecolor'] = BG_COLOR
plt.rcParams['savefig.facecolor'] = BG_COLOR
plt.rcParams['savefig.transparent'] = False

# Dla czytelności na ciemnym tle (bez narzucania palety danych)
plt.rcParams['text.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['grid.color'] = 'white'


In [ ]:
def make_frame_2d(out_dir: Path, k: int, t: float, u: np.ndarray, times: np.ndarray,
                  mass: np.ndarray, dr: np.ndarray, max_u: float) -> Path:
    """
    Tworzy klatkę wideo w układzie 3-panelowym.
    Zwraca ścieżkę do zapisanego pliku PNG.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    fig = plt.figure(figsize=(12, 5))
    fig.patch.set_facecolor(BG_COLOR)

    # Panel 1: Mapa gęstości (2D)
    ax0 = fig.add_subplot(1, 3, 1)
    ax0.set_facecolor(BG_COLOR)

    im = ax0.imshow(u.T, origin='lower', vmax=max_u, interpolation='nearest', cmap='magma')
    ax0.set_title(f'Cell density\nDay {t:.2f}')
    ax0.set_xlabel('x [mm]')
    ax0.set_ylabel('y [mm]')
    ax0.set_xticks([])
    ax0.set_yticks([])

    cbar = fig.colorbar(im, ax=ax0, fraction=0.046, pad=0.04)
    cbar.ax.set_facecolor(BG_COLOR)
    cbar.ax.tick_params(colors='white')

    # Panel 2: Dawka w czasie
    ax1 = fig.add_subplot(1, 3, 2)
    ax1.set_facecolor(BG_COLOR)
    ax1.plot(times, dr, color=COLORS.get('dose', 'red'), lw=1.5)
    ax1.set_title('Dose rate')
    ax1.set_xlabel('Day')
    ax1.set_ylabel('Gy/day')
    ax1.set_xlim(0, float(times[-1]) + 1)
    ax1.grid(True, alpha=0.3)

    # Panel 3: Masa całkowita
    ax2 = fig.add_subplot(1, 3, 3)
    ax2.set_facecolor(BG_COLOR)
    ax2.plot(times, mass, color=COLORS.get('pde', 'blue'), lw=2)
    ax2.set_title('Total tumor mass')
    ax2.set_xlabel('Day')
    ax2.set_ylabel('Sum(u)')
    ax2.set_xlim(0, float(times[-1]) + 1)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()

    fname = out_dir / f'frame_{k:05d}.png'
    plt.savefig(fname, dpi=100, facecolor=fig.get_facecolor(), transparent=False)
    plt.close(fig)
    return fname


In [ ]:
# =============================
# KONFIGURACJA SYMULACJI
# =============================
cfg = SimConfig(
    T_days=60.0,
    dt_day=0.05,
    grid=Grid(L_mm=50.0, N=100),
    # Reszta domyślna jak w oryginale: D=0.137, rho=0.0274,
    # 30 frakcji po 2 Gy, start dzień 10 (o ile tak jest w SimConfig).
)

# Parametry wizualizacji
FRAME_EVERY = 5
FPS = 10

print('--- SYMULACJA 1:1 (LAB01 REPRODUCTION) ---')
print('Wyniki:', OUTPUT_DIR)


In [ ]:
# =============================
# URUCHOMIENIE SYMULACJI + GENEROWANIE KLATEK
# =============================

# Hook do zapisu klatek
def viz_hook(k, t, u, times, mass, dr, max_u):
    if k % FRAME_EVERY == 0:
        if (k // FRAME_EVERY) % 10 == 0:
            print(f'Generowanie klatki dla t={t:.2f}...')
        make_frame_2d(FRAMES_DIR, k, t, u, times, mass, dr, max_u)

print('Start symulacji 2D...')
out = run_2d_simulation(cfg, save_hook=viz_hook)

print('Symulacja zakończona.')


In [ ]:
# =============================
# SKŁADANIE GIF
# =============================

gif_path = OUTPUT_DIR / 'simulation_2d.gif'

print('Składanie GIFa...')
files = sorted(FRAMES_DIR.glob('frame_*.png'))

images = [imageio.imread(str(p)) for p in files]
imageio.mimsave(str(gif_path), images, fps=FPS)

print('Gotowe! Zapisano:', gif_path.resolve())


In [ ]:
# Podgląd ostatniej klatki (opcjonalnie)

if len(list(FRAMES_DIR.glob('frame_*.png'))) > 0:
    last = sorted(FRAMES_DIR.glob('frame_*.png'))[-1]
    import matplotlib.image as mpimg
    img = mpimg.imread(str(last))
    plt.figure(figsize=(6, 3))
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('Brak zapisanych klatek do podglądu.')
